In [1]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
import numpy as np
import re
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [3]:
all_features = ["Language", "tweet_timestamp", "enaged_with_user_id", "engaging_user_id", "engagee_follows_engager",\
                "reply_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [7, 8, 9, 14, 19, 20], sep="\x01")

train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()

In [7]:
train['reply_bool'] = train.reply_timestamp.fillna(0)
train.loc[train.reply_bool != 0.0, 'reply_bool'] = 1.0

In [5]:
train_set['reply_bool'] = train_set.reply_timestamp.fillna(0)
train_set.loc[train_set.reply_bool != 0.0, 'reply_bool'] = 1.0

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
val_set['reply_bool'] = val_set.reply_timestamp.fillna(0)
val_set.loc[val_set.reply_bool != 0.0, 'reply_bool'] = 1.0

In [11]:
val_set

,level_0,index,Language,tweet_timestamp,enaged_with_user_id,engaging_user_id,engagee_follows_engager,reply_timestamp,reply_bool
0,133267715,11908278,125C57F4FA6D4E110983FB11B52EFD4E,1581486678,D75AAE3CCFA44BD84C35942CA4D03E56,A4B1828B571271073C68DA93D1465484,False,NaN,0.0
1,133267716,54640167,125C57F4FA6D4E110983FB11B52EFD4E,1581486678,C7677DF4D001A22F3B969B2ABAB1C179,F3A4C8EB81ADF3685E8F693037808A2C,False,NaN,0.0
2,133267717,102671906,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581486678,46AA727F36DE4EC3CAD7AC2C312421C3,8BD011AB59AC04491687749245009C60,True,NaN,0.0
3,133267718,54937609,975B38F44D65EE42A547283787FF5A21,1581486678,258600B339EB2CBB1006073A145B9CFC,F7BFBA56274FCBBBB76BDD36D0D6495B,True,NaN,0.0
4,133267719,38246969,125C57F4FA6D4E110983FB11B52EFD4E,1581486678,3CE0AF4B64BC1FA67EFA4E98B38CDF08,10F6E035959B9AC156CED11E0FAED566,False,NaN,0.0
...,...,...,...,...,...,...,...,...,...
14807518,148075233,11678309,22C448FF81263D4BAF2A176145EE9EAD,1581551999,63A88B9A7264F35CBFE2C9DFFB428F36,A18266F8C443499B212E666BD19536B7,False,NaN,0.0
14807519,148075234,38584247,ECED8A16BE2A5E8871FD55F4842F16B1,1581551999,BCDC7E16708AC5B155BA52460DFF687F,159C36A1D94F654C5BC48F40D0D76AF9,False,NaN,0.0
14807520,148075235,55912866,22C448FF81263D4BAF2A176145EE9EAD,1581551999,63A88B9A7264F35CBFE2C9DFFB428F36,053D77B11E4B8D6A514715BDA396295D,False,NaN,0.0
14807521,148075236,124669351,22C448FF81263D4BAF2A176145EE9EAD,1581551999,63A88B9A7264F35CBFE2C9DFFB428F36,BC210F06D8397A7C4CD396152136F10C,False,NaN,0.0


In [12]:
train[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'reply_bool']].to_csv('data/train.csv', index = False, sep = '\t')

In [13]:
train_set[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'reply_bool']].to_csv('data/train_set.csv', index = False, sep = '\t')

In [14]:
val_set[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'reply_bool']].to_csv('data/val_set.csv', index = False, sep = '\t')

In [3]:
all_features = ["Language", "tweet_timestamp", "enaged_with_user_id", "engaging_user_id", "engagee_follows_engager",\
                "reply_timestamp"]

test = pd.read_csv("s3://recsys-challenge-2020/val.tsv", encoding="utf-8",
                    names = all_features, usecols= [7, 8, 9, 14, 19, 20], sep="\x01")
test['reply_bool'] = test.reply_timestamp.fillna(0)
test.loc[test.reply_bool != 0.0, 'reply_bool'] = 1.0
test[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'reply_bool']].to_csv('data/test.csv', index = False, sep = '\t')

In [10]:
mask = (test.reply_bool == 0.0)

In [11]:
len(test[mask])

15127684

In [12]:
len(test)

15127684

### Just write the Target for reply

In [2]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
import numpy as np
import re
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [5]:
all_features = ["tweet_timestamp", "reply_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [8, 20], sep="\x01")

train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()

train_set['reply_bool'] = train_set.reply_timestamp.fillna(0)
train_set.loc[train_set.reply_bool != 0.0, 'reply_bool'] = 1.0
val_set['reply_bool'] = val_set.reply_timestamp.fillna(0)
val_set.loc[val_set.reply_bool != 0.0, 'reply_bool'] = 1.0

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
train_set[['reply_bool']].\
to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f1/reply/train_set_target.csv',\
       index = False, header = False)

In [7]:
val_set[['reply_bool']].\
to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f1/reply/val_set_target.csv',\
       index = False, header = False)